# MIE562 Project - Truck Scheduling - MIP Model

**Team 5 - Dylan Camus, Ryan Do, Fan Jia, Matheus Magalhaes, Sugumar Prabhakaran**

**Date: 8 Nov 2020**

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB, quicksum, max_
from random import randint, seed

seed(0)

## Introduction

### 1. Sets
       
   * $j \in J$: A carrier, $j$, in a set of carriers, $J$: 
       * Ex. $J = \left \{1,2,3,\ldots,j  \right \}$
       
   * $|J|$ = n: number of carriers
       
   * $k_{j}$: number of containers for each carrier $j$:    
   
   * $k \in K$: A container, $k$ in a set of containers, $K$: 
       * Ex. $K = \left \{1,2,3... \right \}$   

   * $c \in C$: A chassis, $c$, in a set of chassis, $C$: 
       * Ex. $C = \left \{1,2,3,\ldots,c  \right \}$
 
   * $L = \left \{1,2,3\right\}$: is the set of travel legs, where:
       * $1$ : terminal to transloading facility leg
       * $2$ : terminal to stack leg
       * $3$ : stack to transloading facility leg

In [2]:
# instance intialization

# input: number of arriers, number of chassis, list of containers with release dates, carrier and priority.
nJ = 2
nC = 2
containers = [(3,0,0), (3,0,0), (3,0,0), (12,0,0)] # (release date, carrier, priority)
# containers = [(3, 0, 1)] # (release date, carrier, priority)

# construct sets
J = list(range(nJ))
C = list(range(nC))
K = list(range(len(containers)))
Y = [[0 for _ in K] for _ in J]
L = [1, 2, 3]

for k in K:
    Y[containers[k][1]][k] = 1

print("\nCarriers J:", J)
print("Containers K:", K)
print("Chassis C:", C)
print("Legs L:", L)
print("Set of Binary Variables Y:", Y)


Carriers J: [0, 1]
Containers K: [0, 1, 2, 3]
Chassis C: [0, 1]
Legs L: [1, 2, 3]
Set of Binary Variables Y: [[1, 1, 1, 1], [0, 0, 0, 0]]


In [3]:
# create a list of tuples containing every iteration of k,l,c
A = [(k,l,c) for k in K for l in L for c in C]

### 2. Parameters

   * $\Phi_{l}$  : Time duration of trip on leg l [hours] 
   * $D_{ll'}$: Delay required for a chassis to start leg l' after completing l (including travel) [days]
   * $M$     : Some large number
   * $R_k$   : Release date for container $k$ [days]
   
   * $T_j^{''}$: free period before demurrage cost for carrier j at terminal [days]
   
   * $T_j$: demurrage cost/unit time for carrier j at terminal [($/days)/container]
   
   * $S$     : fixed cost at stack [$/container]
   
   * ${S}'$  : variable cost at stack [($/day)/container]
   
   * $\delta_j^{''}$: free period before detention cost for containers of carrier j [days]
   
   * $\delta_j$: detention cost/unit time for containers of carrier j [($/days)/container]
   
   * $\rho_k$ : priority factor of container k (higher value is higher priority)
   * $p_{k}$  : processing time at transload facility for container k [days]
   
   * $Y_{jk}$ : binary variable = 1 if container $k$ belongs to carrier $j$
   

In [4]:
#Set Parameters

Phi = {1: 6, 2: 2, 3: 5} # time duration of leg l in hours #
D = {1:{1: 0, 2: 0, 3: Phi[2]}, 
     2:{1: Phi[2], 2: Phi[2], 3: 0}, 
     3:{1: 0, 2: 0, 3: Phi[2]}} # have to hard code this- unless we had a formulation for some generic graph of legs which isn't necessary. 

M = 10000000                           # some large number
R = {k: containers[k][0] for k in K}       # release time for container k [hours]

# free period before demurrage cost for containers from carrier j at terminal [hours]
T_prime = {j: 1 for j in J}
# demurrage cost per container of carrier j after free period [($/hour)/container]
T = {j: 100 for j in J}

S = 200                            # $200 fixed cost at stack
S_prime = 20                       # $20/day/container variable cost at stack

# free period before detention cost for containers from carrier j [hours]
Delta_prime = {j: 1 for j in J} 
# detention cost per container of carrier j [($/hour)/container]
Delta = {j: 100 for j in J}

#Y is calculated above as part of set generation
rho = {k: containers[k][2] for k in K} # priority factor for k
p = {k: 5 for k in K}       # processing time at transload facility for k

In [5]:
#Create a new model
model = gp.Model("Truck_Scheduling")

Using license file C:\Users\doryan\gurobi.lic
Academic license - for non-commercial use only


### 3. Decision Variables

   * Binary Variable: $x_{klc}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if}\ \text{container k travels leg l on chassis c} \\
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $
   * Start time $s$ of container $k$ on leg $l$ on chassis $c$: $s_{klc}$, where  $s_{klc} \geq 0$
   * $z_{klk^{'}l^{'}c} = 1$ if the transport job of container $k^{'}$ on chassis c along leg $l^{'}$ is scheduled after the transport job of container $k$ on chassis $c$ along leg $l$.


In [6]:
#Create decision variables
x = model.addVars(A, vtype=GRB.BINARY, name="x_klc")
s = model.addVars(A, vtype=GRB.INTEGER, name="s_klc")
z = model.addVars(K, L, K, L, C, vtype=GRB.BINARY, name="z_klk'l'c")

### 4. Objective Function

We want to minimize the Total Cost ($c$) = (Demurrage cost}) + (Detention cost) + (Stack cost) + (Priority penalty cost)
   
   * **objective function:** $min$ $(c)$
   
   * $c = c_{dem} + c_{det} + c_{stk} + c_{pri}$

**(1) Demurrage Cost:**

The Demurrage Cost ($c_{dem}$) refers to cost associated with keeping containers in the terminal beyond the free period allotedin days ($T_{j}^{''}$) that is different for each carrier.

If the start time for a container ($s_{klc}$) is higher than the release date ($R_{k}$) + the free period ($T_{j}^{''}$), there will be a variable cost per extra day ($T_{j}$) per container.
   
   * $c_{dem} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{1,2\}}\sum_{j \in J}(T_j)\cdot max\left [(s_{klc} - R_k - T_{j}^{''})\cdot (x_{klc}\cdot (Y_{jk}), 0  \right ]$
   
**(2) Detention Cost**

The terminal also charges a detention cost ($c_{det}$) for containers that are not returned to the terminal for processing before return to the carrier.  Similar to demurrage, there is a free period before which the containers must be returned to the terminal ($\delta_{j}^{''}$).

If the start time for a container ($s_{klc}$) and the total travel time of a container back to the terminal including processing time ($\phi_{kl}$) is greater than the release date ($R_{k}$) + the free period ($\delta_{j}^{''}$), there will be a variable cost per extra day ($T_{j}$) per container.

   * $c_{det} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{1,3\}}\sum_{j \in J}(\delta_j)\cdot max\left [(s_{klc}+ \phi_{l}+p_{k} - (R_k+\delta_{j}^{''}))\cdot (x_{klc}\cdot (Y_{jk}), 0  \right ]$

**(3) Stack Storage Cost**

The stack cost ($c_{stk}$) is the sum of the fixed cost percontainer ($S$) and the variable cost per day per container ($S^{'}$) x the number of days that a container is in the stack, which is the difference between leg 3 and leg 2 start times ($s_{klc}$).

   * $c_{stk} = \sum_{k \in K}\sum_{c \in C}x_{k2c}\cdot (S + S'(s_{k3c} - (s_{k2c} + \phi_{2})))$

**(4) Priority Penalty Cost**

The priority penalty cost ($c_{pri}$) is applied for every day that a container sits in the terminal beyond the release day ($R_{k}$) and every day spent at the stack ($s_{k3c}-s_{k2c}$).  Since the priority factor per container ($\rho_{k}$) is higher for higher priority containers, the associated penalty costs will be higher.

   * $c_{det} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{1,2\}}\rho_k\cdot((s_{klc} - R_k)+(s_{k3c} - (s_{k2c} + \phi_{2})))\cdot x_{klc}$

In [7]:
# following variables are for the new demurrage and detention cost
c_dem2 = model.addVars(K, L, C, J, vtype=GRB.INTEGER, name="demurrage cost2")
c_det2 = model.addVars(K, L, C, J, vtype=GRB.INTEGER, name="detention cost2")

# demurrage cost days max
model.addConstrs(c_dem2[k,l,c,j] >= (s[k,l,c] - R[k] - T_prime[j]) for k in K for c in C for l in [1, 2] for j in J)
model.addConstrs(c_dem2[k,l,c,j] >= 0 for k in K for c in C for l in [1, 2] for j in J)

# detention cost days max
model.addConstrs(c_det2[k,l,c,j] >= (s[k,l,c] + Phi[l] +p[k]- (R[k] + Delta_prime[j])) for k in K for c in C for l in [1, 3] for j in J)
model.addConstrs(c_det2[k,l,c,j] >= 0 for k in K for c in C for l in [1, 3] for j in J)

#Define objective function
demurrage_cost = quicksum(x[k,l,c]*T[j]*Y[j][k]*c_dem2[k,l,c,j] 
                          for k in K for c in C for l in [1, 2] for j in J) 
detention_cost = quicksum(x[k,l,c]*Delta[j]*Y[j][k]*c_det2[k,l,c,j] 
                          for k in K for c in C for l in [1, 3] for j in J)
stack_cost = quicksum(x[k,2,c]*(S + S_prime*(s[k,3,c] - (s[k,2,c] + Phi[2]))) for k in K for c in C)

priority_cost = quicksum(x[k,l,c]*rho[k]*((s[k,l,c] - R[k])+(s[k,3,c] - (s[k,2,c] + Phi[2])))
                for k in K for c in C for l in [1, 2])

model.setObjective(demurrage_cost + detention_cost + stack_cost + priority_cost, GRB.MINIMIZE)
# model.setObjective(0, GRB.MINIMIZE) demurrage_cost + detention_cost + stack_cost + priority_cost

### 5. Constraints

**(1) Resource constraints**

For each chassis, transport jobs can't overlap and chassis (resources) may have to travel between jobs. 

For the case where job $klc$ scheduled after job $k'l'c$, i.e. $s_{klc} > s_{k'l'c}$:

   * $max(M(1-x_{klc}), M(1-x_{k'l'c}), s_{klc} - s_{k'l'c}) \geq \phi_{l'} + p_{k'} + D_{l'l} - M\cdot z_{klk'l'c}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k,k' \in K, l,l'\in L, c \in C$ &nbsp;$s.t.$&nbsp; $(k=k' \neg\wedge l=l') \wedge k\leq k'$
   
For the case where job $k'l'c$ scheduled after job $klc$, i.e. $s_{klc} > s_{k'l'c}$:

   * $max(M(1-x_{klc}), M(1-x_{k'l'c}), s_{k'l'c} - s_{klc}) \geq \phi_{l} + p_{k} + D_{ll'} - M\cdot(1-z_{klk'l'c})$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k,k' \in K, l,l'\in L, c \in C$ &nbsp;$s.t.$&nbsp; $(k=k' \neg\wedge l=l') \wedge k\leq k'$

In [8]:
# # Variables needed to use the max method
# max_resource_func1 = model.addVars(K, L, C, vtype=GRB.INTEGER, name="max resource func1")
# max_resource_func2 = model.addVars(K, L, C, vtype=GRB.INTEGER, name="max resource func2")
# max_resource_func3a = model.addVars(K, L, K, L, C, vtype=GRB.INTEGER, name="max resource func3a")
# max_resource_func3b = model.addVars(K, L, K, L, C, vtype=GRB.INTEGER, name="max resource func3b")
max_resource_a = model.addVars(K, L, K, L, C, vtype=GRB.INTEGER, name="max resource a")
max_resource_b = model.addVars(K, L, K, L, C, vtype=GRB.INTEGER, name="max resource b")

# # our max method consists of 3 parts max_(func1, func2, func3a or 3b)
# # max method cannot handle linear equations as parameters, only variables
# # set each function variable: func1, func2, func3a, func 3b
# model.addConstrs(max_resource_func1[k,l,c] == M * (1 - x[k,l,c]) for k in K for l in L for c in C)
# model.addConstrs(max_resource_func2[kp,lp,c] == M * (1 - x[kp,lp,c]) for kp in K for lp in L for c in C)
# model.addConstrs(max_resource_func3a[k,l,kp,lp,c] == (s[k,l,c] - s[kp,lp,c]) 
#                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k < kp)))
# model.addConstrs(max_resource_func3b[k,l,kp,lp,c] == (s[kp,lp,c] - s[k,l,c]) 
#                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k < kp)))

# # max method can only handle "=="" operation, hence, must create max variables
# # First calculation:  max(func1, func2, func3a)
# # Second calculation: max(func1, func2, func3b)
# model.addConstrs(max_resource_a[k,l,kp,lp,c] == 
#                  max_(max_resource_func1[k,l,c], max_resource_func2[kp,lp,c], max_resource_func3a[k,l,kp,lp,c])
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k < kp)))
# model.addConstrs(max_resource_b[k,l,kp,lp,c] == 
#                  max_(max_resource_func1[k,l,c], max_resource_func2[kp,lp,c], max_resource_func3b[k,l,kp,lp,c])
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k < kp)))

model.addConstrs(max_resource_a[k,l,kp,lp,c] >= M * (1 - x[k,l,c]) 
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
model.addConstrs(max_resource_a[k,l,kp,lp,c] >= M * (1 - x[kp,lp,c]) 
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
model.addConstrs(max_resource_a[k,l,kp,lp,c] >= s[k,l,c] - s[kp,lp,c]
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))

model.addConstrs(max_resource_b[k,l,kp,lp,c] >= M * (1 - x[k,l,c]) 
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
model.addConstrs(max_resource_b[k,l,kp,lp,c] >= M * (1 - x[kp,lp,c]) 
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
model.addConstrs(max_resource_b[k,l,kp,lp,c] >= s[kp,lp,c] - s[k,l,c]
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))


# create constraint for each max variable
# first calculation:  if k,l,c is before kp,lp,c
# second calculation: if kp,lp,c is before k,l,c
model.addConstrs(max_resource_a[k,l,kp,lp,c] >= Phi[lp] + p[kp] + D[lp][l] - M * z[k, l, kp, lp, c] 
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
model.addConstrs(max_resource_b[k,l,kp,lp,c] >= Phi[l] + p[k] + D[l][lp] - M * (1-z[k, l, kp, lp, c])
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))


# model.addConstrs(max_(M * (1 - x[k,l,c]), M * (1 - x[kp,lp,c]), (s[k,l,c] - s[kp,lp,c])) 
#                  >= Phi[lp] + p[kp] + D[lp][l] - M * z[k, l, kp, lp, c] 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k < kp)));

# model.addConstrs(max_(M * (1 - x[k,l,c]), M * (1 - x[kp,lp,c]), (s[kp,lp,c] - s[k,l,c])) 
#                  >= Phi[l] + p[k] + D[l][lp] - M * z[k, l, kp, lp, c] 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k < kp)));

# key error for some reason?

{(0, 1, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3, 1, 0): <gurobi.Constr *Awaiting Model Update*

**(2) Release date constraint**

Start time ($s_{klc}$) must be after release date ($R_k$) for every container that leaves terminal on either leg 1 or leg 2 (ie. $x_{klc}=1$):

   * $s_{klc} \geq R_{k}x_{klc}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k \in K$, $l\in L$, $c \in C$ 

In [9]:
model.addConstrs(s[k,l,c]>= R[k]*x[k,l,c] for k in K for l in L for c in C)

{(0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1, 0): <gurobi.Constr *Awaiting Model Upd

**(3) Containers going through leg 2 must go through leg 3**

Each container k will have a x_klc value of 1 or 0 for both leg 2 and leg 3 depending on if they travel through them or not:

   * $\sum_{c \in C}(x_{k2c}) = \sum_{c \in C}(x_{k3c})$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k \in K$

In [10]:
model.addConstrs(quicksum(x[k,2,c] for c in C)==quicksum(x[k,3,c] for c in C) for k in K)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

**(4) Container must go through leg 1 OR (leg 2 AND 3)**

Each container ($k$) will have $x_klc = 1$ for either leg 1 or leg 2 so the sum of the two must be 1 for all containers ($k$) on all chassis ($c$).

   * $\sum_{c \in C}(x_{k1c}) + \sum_{c \in C}(x_{k2c}) = 1$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$

In [11]:
model.addConstrs(
    (quicksum(x[k,1,c] for c in C) + quicksum(x[k,2,c] for c in C))==1 for k in K
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

**(5) Precedence Constraint: Leg 2 must be before Leg 3**

Each container k will have a $x_{klc}$ value of 1 for either leg 1 or leg 2 so the sum of the two must be 1 for all containers (k) on all chassis (c).

   * $M\sum_{c \in C}(x_{k1c}) + \sum_{c \in C}(s_{k3c}) \geq = \sum_{c \in C}(s_{k2c}) + \Phi_{2}$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$
   

In [12]:
model.addConstrs(
    M*(quicksum(x[k,1,c] for c in C)) + quicksum(s[k,3,c] for c in C) >= 
    (quicksum(s[k,2,c] for c in C) + Phi[2]) for k in K
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

**(6) Bind x=0 to s=0**

When $x_{klc} = 0, s_{klc} = 0 $

   * $Mx_{klc} \geq s_{klc}$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$, $l \in L$, $c \in C$

In [13]:
model.addConstrs(M*x[k,l,c] >= s[k,l,c] for k in K for l in L for c in C)

{(0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1, 0): <gurobi.Constr *Awaiting Model Upd

**(7) Domain of $s_{klc}$**

The start time ($s_{klc}$) must be greater than 0:

   * $s_{klc} \geq 0$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$, $l \in L$, $c \in C$

In [14]:
model.addConstrs(s[k,l,c] >=0 for k in K for l in L for c in C)

{(0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1, 0): <gurobi.Constr *Awaiting Model Upd

In [15]:
model.optimize()

for k in K:
    for l in L:
        for c in C:
            print(x[k,l,c], s[k,l,c])

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1460 rows, 1008 columns and 3176 nonzeros
Model fingerprint: 0x8289b24d
Model has 48 quadratic objective terms
Variable types: 0 continuous, 1008 integer (312 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+02, 2e+02]
  QObjective range [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective 1.600000e+12
Presolve removed 1443 rows and 992 columns
Presolve time: 0.02s
Presolved: 35 rows, 38 columns, 86 nonzeros
Presolved model has 12 SOS constraint(s)
Found heuristic solution: objective 2.000000e+11
Variable types: 0 continuous, 38 integer (12 binary)

Root relaxation: objective -1.999968e+08, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2.000e+08    0    2 2.000

# 